<a href="https://colab.research.google.com/github/wlgns8469/Pytorch_Tutorial/blob/main/mnist_nn_dropout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

In [21]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100
drop_prob = 0.3

In [8]:
# MNIST dataset
mnist_train = dsets.MNIST(root = 'MNIST_data/', train = True, transform = transforms.ToTensor(), download = True)
mnist_test = dsets.MNIST(root = 'MNIST_data/', train = False, transform = transforms.ToTensor(), download = True)



In [9]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset = mnist_train, batch_size = batch_size, shuffle = True, drop_last= True)

In [22]:
# nn layers
linear1 = torch.nn.Linear(784, 512, bias=True)
linear2 = torch.nn.Linear(512, 512, bias=True)
linear3 = torch.nn.Linear(512, 512, bias=True)
linear4 = torch.nn.Linear(512, 512, bias=True)
linear5 = torch.nn.Linear(512, 10, bias=True)
relu = torch.nn.ReLU()
dropout = torch.nn.Dropout(p=drop_prob)

In [23]:
# xavier initialization
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)
torch.nn.init.xavier_uniform_(linear4.weight)
torch.nn.init.xavier_uniform_(linear5.weight)

Parameter containing:
tensor([[ 0.0162,  0.0993, -0.0999,  ...,  0.0987,  0.1026,  0.0644],
        [-0.0315,  0.0519,  0.0047,  ..., -0.0434, -0.0674, -0.0864],
        [ 0.0205, -0.0268,  0.0343,  ..., -0.0204,  0.0603, -0.0989],
        ...,
        [ 0.0327,  0.0965, -0.0909,  ..., -0.1048, -0.0263, -0.0621],
        [-0.0821,  0.0444,  0.0772,  ...,  0.0694, -0.0743, -0.0347],
        [ 0.0247, -0.0514, -0.0832,  ..., -0.0021, -0.0081, -0.0917]],
       requires_grad=True)

In [25]:
# model
model = torch.nn.Sequential(linear1, relu, dropout,
                            linear2, relu, dropout,
                            linear3, relu, dropout,
                            linear4, relu, dropout,
                            linear5).to(device)

In [26]:
# define cost/loss & optimizer

criterion = torch.nn.CrossEntropyLoss().to(device) #softmax is internally computed
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)


In [28]:
total_batch = len(data_loader)
model.train()    # set the model to train mode (dropout=True)

for epoch in range(training_epochs):
  avg_cost = 0

  for X, Y in data_loader:
    # reshape input image into [batch_size by 784]
    # label is not one-hot encoded

    X = X.view(-1,28*28).to(device)
    Y = Y.to(device)

    optimizer.zero_grad()
    hypothesis = model(X)
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    avg_cost += cost/total_batch

  print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 0.118606322
Epoch: 0002 cost = 0.099474385
Epoch: 0003 cost = 0.087634511
Epoch: 0004 cost = 0.074999295
Epoch: 0005 cost = 0.071235977
Epoch: 0006 cost = 0.067324884
Epoch: 0007 cost = 0.058357265
Epoch: 0008 cost = 0.054274853
Epoch: 0009 cost = 0.053862140
Epoch: 0010 cost = 0.051085413
Epoch: 0011 cost = 0.049557596
Epoch: 0012 cost = 0.045947447
Epoch: 0013 cost = 0.038892940
Epoch: 0014 cost = 0.043740097
Epoch: 0015 cost = 0.040953036
Learning finished


In [29]:
# Test the model useing test sets

with torch.no_grad():

  model.eval()    # set the model to evaluation mode (dropout=False)
  
  X_test = mnist_test.test_data.view(-1,28*28).float().to(device)
  Y_test = mnist_test.test_labels.to(device)

  prediction = model(X_test)
  correct_prediction = torch.argmax(prediction, 1)== Y_test
  accuracy= correct_prediction.float().mean()
  print('Accuracy: ', accuracy.item() )

  # Get one and predict
  r= random.randint(0, len(mnist_test) - 1)
  X_single_data = mnist_test.test_data[r:r+1].view(-1, 28*28).float().to(device)
  Y_single_data = mnist_test.test_labels[r:r+1].to(device)

  print('Label: ', Y_single_data.item())
  single_prediction = model(X_single_data)
  print('Prediction: ', torch.argmax(single_prediction,1).item())


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:67: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:57: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy:  0.9800999760627747
Label:  5
Prediction:  5
